# Assignment 4 
# Part II: Semi-supervised Low-level Segmentation
## (40% weight of the assignment)

NOTE: Your code should "recreate" representative results shown for graph cuts in Topic 9, but they do not have to be identical.

## Problem 1
### Implement interactive seed-based segmentation using s/t graph cut.
#### A basic seed-interactive GUI "GraphCutsPresenter" is available (implemented in "asg1.py"). The starter code below uses it. The presenter allows to add seeds (left and right mouse clicks for object and background seeds) and displays these seeds over the image. However, instead of proper graph cut segmentation the provided code displays some fixed binary mask (just as an example of a mask). This "fixed" mask should be replaced by the output of an interactive image segmentation method based on minimum s/t graph cut respecting the hard constraints marked by the user seeds. You can use an existing python library for computing minimum s/t cuts on arbitrary graphs (run "$\text{pip install PyMaxFlow}$" in Anaconda Prompt, see <a href="http://pmneila.github.io/PyMaxflow/maxflow.html">documentation</a>). You can use this library to build a weighted graph based on selected image and user-entered seeds.
## Sub-problem 1.1 (70% weight of the problem)
#### As a <font color="red"> first milestone </font>, you should implement the most basic version of the graph cut algorithm for interactive image segmentation based on user seeds (see slides 9-12, topic 9B) that minimizes the following weak-supervision loss (see slides 13-18, topic 9B)  $$ -\sum_{p\in \Omega_{\mathcal L}} \log S_p^{y_p} \;\;+\;\;\lambda \sum_{pq\in N} w_{pq}\,[S_p \neq S_q] $$ combining the supervision loss, a.k.a. seed loss (hard constraints for user-defined labels), and basic pair-wise regularization loss with "contrast-weights" $w_{pq} = \exp\frac{-\|I_p-I_q\|^2}{\sigma^2}$ for 4-connected grid neighborhood. Note that the scalar (hyper-parameter) $\lambda$ controlling the regularization strength can be integrated into edge weights $\tilde{w}_{pq} = \lambda \exp\frac{-\|I_p-I_q\|^2}{\sigma^2}$ (parameter $\lambda$ is primarily needed for the second milestone below). Terminal "t-links" for seed-points should make sure that such graph nodes can not be severed from the corresponding terminals. You have to use "large-enough" finite cost t-links to enforce hard-constraints (user seeds) since "infinity" weight is not possible to implement. One can argue that $N\cdot \max \{\tilde{w}_{pq}\}\equiv N\lambda$ (where N is the number of neighbors at each point) is sufficient.
## Sub-problem 1.2 (30% weight of the problem)
#### Once the first version of your interactive segmentation above is implemented and <u> tested</u>, your <font color="red">second milestome </font> is to add color-based negative log-likelihoods (NLL), see slides 52 or 57 in topic 9B, into the total loss $$ -\sum_p \log Pr(I_p\,|\,\theta_{S_p}) \;\;-\;\;\sum_{p\in \Omega_{\mathcal L}} \log S_p^{y_p} \;\;+\;\;\lambda\sum_{pq\in N} w_{pq}\,[S_p \neq S_q] $$ where two distinct color distributions $\Pr(I|\theta_1)$ and $\Pr(I|\theta_0)$ for two segments (object and backgound) should be estimated as GMMs from the corresponding seeds, as follows. As discussed in topic 9A (slides 60-61), GMM is a standard way to efficiently estimate arbitrary multi-modal probability density functions using relatively few parameters. Note that this assignment is not using GMMs to cluster seeds or any other pixels. We estimate one set of parameters $\theta_1$ for the GMM probability denisty function $\Pr(I|\theta_1)$ for RGB colors at pixels with "red" seeds (treated as a sample of object features), and the second set of parameters $\theta_0$ of the GMM probability density function $\Pr(I|\theta_0)$  for RGB colors at pixels with "blue" seeds (sample of background features). You do not need to implement EM algorithm (slide 67, topic 9A) for estimating $\theta_1$ and $\theta_0$, just use "GaussianMixture" function from the standard "mixture" library in "sklearn"; mixtures of six Gaussian modes ($K=6$) should suffice for each GMM density. Note that once you estimate two color distributions/densities (from seeds), then you can evaluate two likelihoods $\Pr(I_p|\theta_1)$ and $\Pr(I_p|\theta_0)$ at any pixel $p$, as required for the color log-likelihoods term in the total loss above.

## NOTES/HINTS for both sub-problems
#### NOTE 1: max-flow/min-cut libraries are typically more efficient when using integer edge weights in a relatively small range. You can use integer-weighted graph where edge weights are discretized/truncated values of your edge-weighting function.
#### NOTE 2: Test different values of "regularization parameter" $\lambda$ controlling relative weight of the regularization term versus the likelihoods terms.
#### NOTE 3: Play with parameter $\sigma$ for exponential n-link weighting function in $w_{pq}\propto \exp\frac{-\|I_p-I_q\|^2}{\sigma^2}$ using intensity differences/contrast between two pixels. Test different values of  $\sigma$. Show 2-3 representative results (in different cells). Use markdown cell to discuss your observations, if any. If you can suggest some specific way of selecting some $\sigma$ adaptively to each image, provide a brief technical motivation for it.
#### NOTE 4: You can build either 4 or 8 nearest-neighbors (NN) grid.
#### NOTE 5:  One numerical issue you may face are intensities/colors $I$ such that $Pr(I|\theta_k)\approx0$ where pixels with $I$ may still be present in segment $k$, but this intensity was not represented in the sample (seeds) used to estimate the distribution for segment $k$. This implies near-infinity log-likelihoods forbidding any pixel of color $I$ from segment $k$. Another issue could be that seeds for category $k$ may mistakenly include some pixels from the other category (e.g. if the user made errors when placing seeds). The core issue here is robustness of the log-likelihhood loss with respect to imperfect distributions/densities. There is a standard robustification trick (also used for log-likelihood losses in supervised network training to address labels with errors). The general idea is to modify (both segments) likelihoods as follows:  $$P'(I|\theta_k) \;=\; \gamma\, P(I|\theta_k) + \frac{1-\gamma}{\|RGB\|} $$ which can be derived as a mixture model for two distributions: $P$ and the uniform distribution over color space where $\|RGB\|$ is its cardinality. Essentially, robust modification of the log-likeliohood loss boils down to 
#### $$-log P(I|\theta_k) \;\;\;\;\;\;\rightarrow\;\;\;\;\;\;\; -\log \left( P(I|\theta_k) + \epsilon \right)$$ 
#### where $\epsilon$ is an additional "robustness" parameter that bounds/truncates near-infinite log-likelighood penalties.

#### NOTE 6: (Creating n-links) Here is an illustration of one way of creating (undirected) n-links for the simplest 4-connected grid used in all examples of topic 9. First, you should compute two 2D arrays n_right and n_below (same shape as your image) where each element corresponding to pixel p is a weight wpq of n-link from this pixel to its neighbor q on the right or below (respectively). The weights should be computed according to the provided formula. You can use np.roll, np.exp, and other numpy functions to compute such arrays avoiding double-for-loops. Then, you can add the corresponding n-links to your graph as follows:

g = maxflow.GraphFloat()

nodeids = g.add_grid_nodes((num_rows, num_cols))

structure_x = np.array([[0, 0, 0],
                        [0, 0, 1],
                        [0, 0, 0]])
                        
structure_y = np.array([[0, 0, 0],
                        [0, 0, 0],
                        [0, 1, 0]])

g.add_grid_edges(nodeids, weights=n_right, structure=structure_x, symmetric=True)

g.add_grid_edges(nodeids, weights=n_below, structure=structure_y, symmetric=True)

#### NOTE 7: (Creating t-links) Assume you already computed two 2D arrays t_sink and t_source (same shape as your image) where each element corresponding to pixel p is a weight of t-link from this pixel to the source or sink terminals (respectively). When computing such arrays, you should avoid double for-loops. Then, t-links can be created as below.

g.add_grid_tedges( nodeids, t_source, t_sink )

#### This function increments the weight of existing t-links, which are automatically created with each node (initialized to zero). You can call this function for the same nodes many times before or after the max-flow is computed. To update the solution after each modification of edge weights, you should call max-flow again. HINT:  if you need to change t-links to be equal to some particular "new_weights", you may need to create additional arrays storing current values of t-links and update t-links as follows:

tlinks_source = np.zeros((num_rows,num_cols))

tlinks_sink   = np.zeros((num_rows,num_cols))

...

...

g.add_grid_tedges( nodeids, new_weights_source - tlinks_source, new_weights_sink - tlinks_sink)

tlinks_source = new_weights_source

tlinks_sink   = new_weights_sink

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

# loading standard library for GMM estimation
from sklearn.mixture import GaussianMixture

# loading standard maxflow library (graph cuts)
# Before importing maxflow first time, you must "pip install PyMaxflow" in Anaconda Prompt (see https://pypi.org/project/PyMaxflow/) 
import maxflow

# loading custom module (requires file asg1.py in the same directory as the notebook file)
from asg1_error_handling import Figure, GraphCutsPresenter

In [2]:
class MyGraphCuts:
    bgr_value = 0
    obj_value = 1
    none_value = 2
    
    def __init__(self, img, sigma=0.5, lambd=0):
        self.img = img
        self.sigma = sigma                  # controls contrast sensitivity for n-link weights
        self.lambd = lambd                  # boundary regularization parameter
        
        self.fig = Figure()
        self.pres = GraphCutsPresenter(img, self)
        self.pres.connect_figure(self.fig)

        self.num_rows = img.shape[0]
        self.num_cols = img.shape[1]

    def run(self):
        self.fig.show()

    # fit GMMs to seeds
    def fit_gmms(self, seed_mask):
        # extract seed pixels for object vs background
        pixels = self.img.reshape(-1, 3)
        obj_pixels = pixels[seed_mask.flatten() == self.obj_value]
        bgr_pixels = pixels[seed_mask.flatten() == self.bgr_value]
        
        # fit GMMs only if there are enough samples for object/background
        self.obj_gmm = GaussianMixture(n_components=6, random_state=0) if len(obj_pixels) >= 2 else None
        self.bgr_gmm = GaussianMixture(n_components=6, random_state=0) if len(bgr_pixels) >= 2 else None

        if self.obj_gmm: self.obj_gmm.fit(obj_pixels)
        if self.bgr_gmm: self.bgr_gmm.fit(bgr_pixels)
    
    # compute source and sink terminal links based on modified GMM log-likelihoods
    def compute_t_links(self, seed_mask):
        self.fit_gmms(seed_mask)                # fit GMMs
        pixels = self.img.reshape(-1, 3)
        
        # default uniform log-likelihoods for unseen colors
        default_log_likelihood = np.log(1.0 / 256**3)
        obj_log_likelihood = np.full(pixels.shape[0], default_log_likelihood)
        bgr_log_likelihood = np.full(pixels.shape[0], default_log_likelihood)

        # update log-likelihoods using GMMs, if available
        if self.obj_gmm: obj_log_likelihood = self.obj_gmm.score_samples(pixels)
        if self.bgr_gmm: bgr_log_likelihood = self.bgr_gmm.score_samples(pixels)
        
        # Trick to avoid numerical instability and be robust in user error:
        #   Combine likelihood with uniform distribution
        #       P'(I|theta_k) = gamma * P(I|theta_k) + (1 - gamma) / |RGB|
        #   Yielding the modified log-liklihood:
        #       -log P'(I|theta_k) = -log(gamma * P(I|theta_k) + (1 - gamma) / |RGB|)

        gamma = 0.75                           # weight given to GMM likelihood (lower = more uniform influence = more robust)
        uniform_term = (1 - gamma) / (256**3)   

        obj_cost = -np.log(gamma * np.exp(obj_log_likelihood) + uniform_term)
        bgr_cost = -np.log(gamma * np.exp(bgr_log_likelihood) + uniform_term)

        # reshape costs to image dimensions
        obj_cost = obj_cost.reshape(self.num_rows, self.num_cols)
        bgr_cost = bgr_cost.reshape(self.num_rows, self.num_cols)

        # need to enforce hard constraints for seed pixels.
        # assign high cost to ensure strict assignment (probably should be proportional to max cost)
        seed_weight = 1e+8
        t_source = np.where(seed_mask == self.obj_value, seed_weight, obj_cost)
        t_sink = np.where(seed_mask == self.bgr_value, seed_weight, bgr_cost)

        return t_source, t_sink
    
    # compute n-link weights based on intensity contrast
    def compute_n_links(self):
        # calculate the intensity differences for right and below neighbours
        diff_right = np.roll(self.img, -1, axis=1) - self.img
        diff_below = np.roll(self.img, -1, axis=0) - self.img
        
        # remove wrap-around differences
        diff_right[:, -1, :] = 0
        diff_below[-1, :, :] = 0
        
        # compute weights based on contrast and regularization parameter
        n_right = self.lambd * np.exp(-np.linalg.norm(diff_right, axis=2)**2 / self.sigma**2)
        n_below = self.lambd * np.exp(-np.linalg.norm(diff_below, axis=2)**2 / self.sigma**2)

        return n_right, n_below

    # perform graph cut and return segmentation labels
    def compute_labels(self, seed_mask):
        g = maxflow.Graph[float]()                                          # initialize graph
        nodeids = g.add_grid_nodes((self.num_rows, self.num_cols))          # nodes for each pixel

        n_right, n_below = self.compute_n_links()                           # compute n-links
        
        structure_right = np.array([[0, 0, 0],                              # define structures for right and below neighbors
                                    [0, 0, 1], 
                                    [0, 0, 0]])
        structure_below = np.array([[0, 0, 0],
                                    [0, 0, 0],
                                    [0, 1, 0]])                             # add n-links for neighbors in the graph
        g.add_grid_edges(nodeids, n_right, structure=structure_right, symmetric=True)
        g.add_grid_edges(nodeids, n_below, structure=structure_below, symmetric=True)

        t_source, t_sink = self.compute_t_links(seed_mask)                  # compute and add t-links to the graph
        g.add_grid_tedges(nodeids, t_source, t_sink)

        g.maxflow()                                                         # get segmentation
        segments = g.get_grid_segments(nodeids)

        label_mask = np.where(segments, self.obj_value, self.bgr_value)     # make mask 

        return label_mask

### Notes about the basic graph cut interface:
1. To provide the regional hard constraints (seeds) for object and background segments use left and right mouse clicks (mouse dragging works somewhat too). Use mouse wheel to change the brush size.
2. The seed mask is built by the "GraphCutsPresenter". Each mouse release activates "on_mouse_up" function of the presenter, which asks the linked MyGraphCuts object to "compute_labels" for all pixels
based on the provided seed mask.
3. You should use "PyMaxflow" library (already imported into this notebook if you ran the second cell) to build a weighted graph and to compute a minimum s/t cut defining all pixel labels from the seeds as explain in topic 5.

### Bunny Reference Image

<p align="center">
    <img src="images/bunny.bmp" height="250"/>
</p>

In [3]:
img1 = plt.imread('images/bunny.bmp')

#app = MyGraphCuts(img1, sigma=0.5, lambd=5.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/bunny1a.png" height="300"/>
    <img src="images/Part2Plots/bunny1b.png" height="300"/>
</p>

In [4]:
#app = MyGraphCuts(img1, sigma=5.0, lambd=5.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/bunny2a.png" height="300"/>
    <img src="images/Part2Plots/bunny2b.png" height="300"/>
</p>

In [5]:
#app = MyGraphCuts(img1, sigma=50.0, lambd=5.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/bunny3a.png" height="300"/>
    <img src="images/Part2Plots/bunny3b.png" height="300"/>
</p>

In [6]:
#app = MyGraphCuts(img1, sigma=500.0, lambd=5.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/bunny4a.png" height="300"/>
    <img src="images/Part2Plots/bunny4b.png" height="300"/>
</p>

______________________________

#### Discussion on segmentation using different $\sigma$:

At low values of $\sigma$, segmentation boundaries are highly sensitive to contrasting edges. Boundaries are very sharp and defined but small intensity variations from noise and texture can be seen incorrectly segmented into smaller parts.

As $\sigma$ increases, intensity differences are downweighted less significantly. The segmentation becomes smoother, but boundaries are blurred are the size of segments become larger.

To select $\sigma$ adaptively for each image, we could consider the intensity distribution and contrast within the image. For example, for each pixel 
$p$, compute the mean or median intensity difference with its neighbors. We can then use these local contrast values to estimate an overall scale of intensity variation in the image.

In conclusion, the parameter $\sigma$ influences the segmentation outcome by controlling the contrast sensitivity of n-links. An adaptive strategy, based on local contrast statistics, may provide a robust method to balance edge preservation and smoothness across diverse images.

______________________________

### Show how your interactive segmenter works on more challanging images where there is some overlap between the color-models in the object and background (as in the "lama" image). Compare the results for $\lambda=0$ (no regularization) and for some $\lambda>0$. For convenience, you might want to include $\lambda$ in the list of parameters for the function "MyGraphcuts".

### Lama Reference Image

<p align="center">
    <img src="images/lama.jpg" height="250"/>
</p>

In [7]:
img2 = plt.imread('images/lama.jpg')

#app = MyGraphCuts(img2, sigma=50.0, lambd=0.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/lama1a.png" height="300"/>
    <img src="images/Part2Plots/lama1b.png" height="300"/>
</p>

In [8]:
#app = MyGraphCuts(img2, sigma=50.0, lambd=10.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/lama2a.png" height="300"/>
    <img src="images/Part2Plots/lama2b.png" height="300"/>
</p>

In [9]:
#app = MyGraphCuts(img2, sigma=50.0, lambd=100.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/lama3a.png" height="300"/>
    <img src="images/Part2Plots/lama3b.png" height="300"/>
</p>

In [ ]:
#app = MyGraphCuts(img2, sigma=50.0, lambd=1000.0)
#app.run()

<p align="center">
    <img src="images/Part2Plots/lama4a.png" height="300"/>
    <img src="images/Part2Plots/lama4b.png" height="300"/>
</p>

______________________________

#### Discussion on segmentation using different $\lambda$:

With $\lambda = 0$ (ie. no regularization), segmentation depends solely on the color-based likelihoods, meaning the labels assigned to pixels are determined entirely by the similarity of pixel colors to the GMMs fitted to the seed regions. In the lama image, areas with similar colors in the fur and background were misclassified due to similarity in the color, and a reliance on that data alone.

As $\lambda$ increases, segmentation is encouraged to maintain spatial coherence. Segmentation boundaries become smoother and more regularized, overlapping regions are better resolved, and there is a significant reduction in noise. However, it can lead to oversmoothing and loss to finer details. 

______________________________